# Start

In [1]:
import importlib
import os
import pandas as pd
import json
from datetime import datetime
from pathlib import Path
from io import StringIO

from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

In [2]:
from bsm_multi_agents.agents import utils
importlib.reload(utils)
from bsm_multi_agents.agents.utils import get_tool_result_from_messages,print_resp

In [3]:
cwd = Path.cwd()
project_path = cwd.parent.parent
project_path

PosixPath('/Users/yifanli/Github/model_doc_automation')

# Graph

In [4]:
from langgraph.graph import StateGraph, END, START

from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState
from bsm_multi_agents.agents.data_loader_agent import data_loader_node
from bsm_multi_agents.agents.calculator_agent import calculator_node
from bsm_multi_agents.agents.validator_agent import validator_node

graph = StateGraph(WorkflowState)
graph.add_node("data_loader", data_loader_node)
graph.add_node("calculator", calculator_node)
graph.add_node("validator", validator_node)

graph.add_edge(START, "data_loader")
graph.add_edge("data_loader", "calculator")
graph.add_edge("calculator", "validator")
graph.add_edge("validator", END)

app = graph.compile()

In [5]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
init_state: WorkflowState = {
    "csv_file_path": str(file_path),
    "remaining_steps": 10,
    "messages": [HumanMessage(content=f"Load CSV from: {file_path}")],
}

final_state = app.invoke(
    init_state,
    config={"configurable": {"thread_id": "run-1"}}
)

[data_loader] LLM: ChatOllama (qwen3:8b), Tools: ['csv_loader']
[calculator] LLM: ChatOllama (qwen3:8b), Tools: ['batch_bsm_calculator', 'batch_greeks_calculator', 'sensitivity_test']
[validator] LLM: ChatOllama (qwen3:8b), Tools: ['batch_greeks_validator']


In [6]:
final_state

{'messages': [HumanMessage(content='Load CSV from: /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv', additional_kwargs={}, response_metadata={}, id='585c4af1-3a32-4ebb-ad38-812655cc778a'),
  HumanMessage(content='Load the option data from the CSV file at: /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv\n\nUse the csv_loader tool to read the CSV file. Return the data in JSON format.', additional_kwargs={}, response_metadata={}, id='d8f1cb37-5c61-4376-a41a-0f9b79e6a347'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3:8b', 'created_at': '2025-11-29T22:20:59.800897Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6446706541, 'load_duration': 2375743583, 'prompt_eval_count': 258, 'prompt_eval_duration': 360184916, 'eval_count': 266, 'eval_duration': 3622192497, 'model_name': 'qwen3:8b', 'model_provider': 'ollama'}, id='lc_run--7bf48c55-c893-4923-992d-cc4bee5dd900-0', tool_calls=[{'name': 'csv_loader'

# Data Loader

## Function Level

In [6]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
df = pd.read_csv(file_path)
json_str = df.to_json(orient='records')
result = {"csv_data": json_str}

FileNotFoundError: [Errno 2] No such file or directory: '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv'

In [5]:
json.dumps(result)

'{"csv_data": "[{\\"date\\":\\"2025-09-01\\",\\"S\\":100,\\"K\\":105,\\"T\\":1.0,\\"r\\":0.05,\\"sigma\\":0.2,\\"option_type\\":\\"call\\"},{\\"date\\":\\"2025-09-02\\",\\"S\\":102,\\"K\\":106,\\"T\\":0.9,\\"r\\":0.045,\\"sigma\\":0.19,\\"option_type\\":\\"put\\"},{\\"date\\":\\"2025-09-03\\",\\"S\\":98,\\"K\\":104,\\"T\\":0.8,\\"r\\":0.048,\\"sigma\\":0.21,\\"option_type\\":\\"call\\"},{\\"date\\":\\"2025-09-04\\",\\"S\\":101,\\"K\\":107,\\"T\\":0.7,\\"r\\":0.047,\\"sigma\\":0.18,\\"option_type\\":\\"call\\"},{\\"date\\":\\"2025-09-05\\",\\"S\\":99,\\"K\\":103,\\"T\\":0.6,\\"r\\":0.046,\\"sigma\\":0.22,\\"option_type\\":\\"put\\"},{\\"date\\":\\"2025-09-06\\",\\"S\\":103,\\"K\\":108,\\"T\\":0.5,\\"r\\":0.049,\\"sigma\\":0.2,\\"option_type\\":\\"put\\"},{\\"date\\":\\"2025-09-07\\",\\"S\\":97,\\"K\\":102,\\"T\\":0.4,\\"r\\":0.044,\\"sigma\\":0.23,\\"option_type\\":\\"call\\"},{\\"date\\":\\"2025-09-08\\",\\"S\\":100,\\"K\\":106,\\"T\\":0.3,\\"r\\":0.05,\\"sigma\\":0.19,\\"option_type\\

## Tool Level

In [6]:
from bsm_multi_agents.tools import data_loader_tools
importlib.reload(data_loader_tools)
from bsm_multi_agents.tools.data_loader_tools import csv_loader

In [7]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
csv_loader.invoke(file_path)

'{"csv_data": "[{\\"date\\":\\"2025-09-01\\",\\"S\\":100,\\"K\\":105,\\"T\\":1.0,\\"r\\":0.05,\\"sigma\\":0.2,\\"option_type\\":\\"call\\"},{\\"date\\":\\"2025-09-02\\",\\"S\\":102,\\"K\\":106,\\"T\\":0.9,\\"r\\":0.045,\\"sigma\\":0.19,\\"option_type\\":\\"put\\"},{\\"date\\":\\"2025-09-03\\",\\"S\\":98,\\"K\\":104,\\"T\\":0.8,\\"r\\":0.048,\\"sigma\\":0.21,\\"option_type\\":\\"call\\"},{\\"date\\":\\"2025-09-04\\",\\"S\\":101,\\"K\\":107,\\"T\\":0.7,\\"r\\":0.047,\\"sigma\\":0.18,\\"option_type\\":\\"call\\"},{\\"date\\":\\"2025-09-05\\",\\"S\\":99,\\"K\\":103,\\"T\\":0.6,\\"r\\":0.046,\\"sigma\\":0.22,\\"option_type\\":\\"put\\"},{\\"date\\":\\"2025-09-06\\",\\"S\\":103,\\"K\\":108,\\"T\\":0.5,\\"r\\":0.049,\\"sigma\\":0.2,\\"option_type\\":\\"put\\"},{\\"date\\":\\"2025-09-07\\",\\"S\\":97,\\"K\\":102,\\"T\\":0.4,\\"r\\":0.044,\\"sigma\\":0.23,\\"option_type\\":\\"call\\"},{\\"date\\":\\"2025-09-08\\",\\"S\\":100,\\"K\\":106,\\"T\\":0.3,\\"r\\":0.05,\\"sigma\\":0.19,\\"option_type\\

## Agent Level

In [4]:
from bsm_multi_agents.agents import agent_factory
importlib.reload(agent_factory)
from bsm_multi_agents.agents.agent_factory import built_graph_agent_by_role
from bsm_multi_agents.graph.state import WorkflowState
from bsm_multi_agents.prompts.loader import load_prompt

In [5]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
state = {
    "csv_file_path": file_path
}
prompt_path = os.path.join(project_path, "src/bsm_multi_agents/prompts/data_loader_prompts.txt")

In [6]:
agent_role = 'data_loader'
agent = built_graph_agent_by_role(agent_role)

[data_loader] LLM: ChatOllama (qwen2.5:7b), Tools: ['csv_loader']


In [9]:
csv_path = state.get("csv_file_path")
prompt = load_prompt(prompt_path).format(csv_path=str(csv_path))
msg = HumanMessage(content=prompt)

agent_input = {
    "messages": [msg],
    "remaining_steps": 10
}
result = agent.invoke(
    agent_input,
    config={
        "recursion_limit": 10,
        "configurable": {"thread_id": "run-1"}
    }
)

In [10]:
print_resp(result)

Step 1 - inputs:
   Load the option data from the CSV file at: /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv

Use the csv_loader tool to read the CSV file. Return the data in JSON format.

Step 2 - Agent decide tools used:
   Tool name: csv_loader
   Tool parameters: {'filepath': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv'}

Step 3 - outputs:
   Tool name: csv_loader
   Outputs: {"csv_data": "[{\"date\":\"2025-09-01\",\"S\":100,\"K\":105,\"T\":1.0,\"r\":0.05,\"sigma\":0.2,\"option_type\":\"call\"},{\"date\":\"2025-09-02\",\"S\":102,\"K\":106,\"T\":0.9,\"r\":0.045,\"sigma\":0.19,\"option_type\":\"put\"},{\"date\":\"2025-09-03\",\"S\":98,\"K\":104,\"T\":0.8,\"r\":0.048,\"sigma\":0.21,\"option_type\":\"call\"},{\"date\":\"2025-09-04\",\"S\":101,\"K\":107,\"T\":0.7,\"r\":0.047,\"sigma\":0.18,\"option_type\":\"call\"},{\"date\":\"2025-09-05\",\"S\":99,\"K\":103,\"T\":0.6,\"r\":0.046,\"sigma\":0.22,\"option_type\":\"put\"},{\"date\":\"2

In [11]:
csv_data = get_tool_result_from_messages(result["messages"], "csv_loader")

In [12]:
json.loads(csv_data["result"])['csv_data']

'[{"date":"2025-09-01","S":100,"K":105,"T":1.0,"r":0.05,"sigma":0.2,"option_type":"call"},{"date":"2025-09-02","S":102,"K":106,"T":0.9,"r":0.045,"sigma":0.19,"option_type":"put"},{"date":"2025-09-03","S":98,"K":104,"T":0.8,"r":0.048,"sigma":0.21,"option_type":"call"},{"date":"2025-09-04","S":101,"K":107,"T":0.7,"r":0.047,"sigma":0.18,"option_type":"call"},{"date":"2025-09-05","S":99,"K":103,"T":0.6,"r":0.046,"sigma":0.22,"option_type":"put"},{"date":"2025-09-06","S":103,"K":108,"T":0.5,"r":0.049,"sigma":0.2,"option_type":"put"},{"date":"2025-09-07","S":97,"K":102,"T":0.4,"r":0.044,"sigma":0.23,"option_type":"call"},{"date":"2025-09-08","S":100,"K":106,"T":0.3,"r":0.05,"sigma":0.19,"option_type":"call"},{"date":"2025-09-09","S":104,"K":109,"T":0.2,"r":0.045,"sigma":0.21,"option_type":"put"},{"date":"2025-09-10","S":96,"K":101,"T":0.1,"r":0.048,"sigma":0.2,"option_type":"put"}]'

### main level

In [13]:
from bsm_multi_agents.agents.data_loader_agent import data_loader_node

In [14]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
state = WorkflowState(csv_file_path=file_path)
out = data_loader_node(state)
out

[data_loader] LLM: ChatOllama (qwen2.5:7b), Tools: ['csv_loader']


{'messages': [HumanMessage(content='Load the option data from the CSV file at: /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv\n\nUse the csv_loader tool to read the CSV file. Return the data in JSON format.', additional_kwargs={}, response_metadata={}, id='c17cc3df-bef0-4eda-b631-4c67d7b27dcd'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-11-25T21:06:46.728252Z', 'done': True, 'done_reason': 'stop', 'total_duration': 540636833, 'load_duration': 67770375, 'prompt_eval_count': 256, 'prompt_eval_duration': 70422833, 'eval_count': 36, 'eval_duration': 390163336, 'model_name': 'qwen2.5:7b', 'model_provider': 'ollama'}, id='lc_run--23605455-f6b5-4348-a77f-9e0fbe5761f4-0', tool_calls=[{'name': 'csv_loader', 'args': {'filepath': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv'}, 'id': '76a829f2-f4a6-4dea-99d0-eabb9018d829', 'type': 'tool_call'}], usage_metadata={'input_tokens': 256

# Calculator

In [15]:
from bsm_multi_agents.agents.data_loader_agent import data_loader_node

In [16]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
state = WorkflowState(csv_file_path=file_path)
state = data_loader_node(state)

[data_loader] LLM: ChatOllama (qwen2.5:7b), Tools: ['csv_loader']


## Function Level

In [17]:
from bsm_multi_agents.tools import bsm_calculator_tools
importlib.reload(bsm_calculator_tools)
from bsm_multi_agents.tools.bsm_calculator_tools import _bsm_calculator, _greeks_calculator
from bsm_multi_agents.tools.utils import load_json_as_df

### batch_bsm_calculator

In [18]:
csv_data = state.get("csv_data")
df = load_json_as_df(csv_data)
df.head()

,date,S,K,T,r,sigma,option_type
0,2025-09-01,100,105,1.0,0.050,0.20,call
1,2025-09-02,102,106,0.9,0.045,0.19,put
2,2025-09-03,98,104,0.8,0.048,0.21,call
3,2025-09-04,101,107,0.7,0.047,0.18,call
4,2025-09-05,99,103,0.6,0.046,0.22,put


In [19]:
required_cols = ['option_type', 'S', 'K', 'T', 'r', 'sigma']
missing_cols = [col for col in required_cols if col not in df.columns]
missing_cols


[]

In [20]:
def calc_row(row):
    return _bsm_calculator(
        row['option_type'], 
        row['S'], 
        row['K'], 
        row['T'], 
        row['r'], 
        row['sigma']
    )
df['BSM_Price'] = df.apply(
    calc_row, axis=1
)
result = {"bsm_results": df.to_json(orient='records', date_format='iso')}
json.dumps(result)

'{"bsm_results": "[{\\"date\\":\\"2025-09-01T00:00:00.000\\",\\"S\\":100,\\"K\\":105,\\"T\\":1.0,\\"r\\":0.05,\\"sigma\\":0.2,\\"option_type\\":\\"call\\",\\"BSM_Price\\":8.0213522351},{\\"date\\":\\"2025-09-02T00:00:00.000\\",\\"S\\":102,\\"K\\":106,\\"T\\":0.9,\\"r\\":0.045,\\"sigma\\":0.19,\\"option_type\\":\\"put\\",\\"BSM_Price\\":7.2142383713},{\\"date\\":\\"2025-09-03T00:00:00.000\\",\\"S\\":98,\\"K\\":104,\\"T\\":0.8,\\"r\\":0.048,\\"sigma\\":0.21,\\"option_type\\":\\"call\\",\\"BSM_Price\\":6.415749359},{\\"date\\":\\"2025-09-04T00:00:00.000\\",\\"S\\":101,\\"K\\":107,\\"T\\":0.7,\\"r\\":0.047,\\"sigma\\":0.18,\\"option_type\\":\\"call\\",\\"BSM_Price\\":4.9529618176},{\\"date\\":\\"2025-09-05T00:00:00.000\\",\\"S\\":99,\\"K\\":103,\\"T\\":0.6,\\"r\\":0.046,\\"sigma\\":0.22,\\"option_type\\":\\"put\\",\\"BSM_Price\\":7.3776960425},{\\"date\\":\\"2025-09-06T00:00:00.000\\",\\"S\\":103,\\"K\\":108,\\"T\\":0.5,\\"r\\":0.049,\\"sigma\\":0.2,\\"option_type\\":\\"put\\",\\"BSM_Price

### batch_greeks_calculator

In [21]:
csv_data = state.get("csv_data")
df = load_json_as_df(csv_data)
required_cols = ['option_type', 'S', 'K', 'T', 'r', 'sigma']
missing_cols = [col for col in required_cols if col not in df.columns]
missing_cols

[]

In [22]:
def calc_row(row):
    res = _greeks_calculator(
        row['option_type'], 
        row['S'], 
        row['K'], 
        row['T'], 
        row['r'], 
        row['sigma'], 
    )
    return res
expanded = df.apply(calc_row, axis=1).apply(pd.Series)
result_cols = ['price','delta','gamma','vega','rho','theta']
for col in result_cols:
    if col not in expanded:
        expanded[col] = pd.NA
df = pd.concat([df, expanded[result_cols]], axis=1)

In [23]:
result = {"greeks_results": df.to_json(orient='records', date_format='iso')}
json.dumps(result)

'{"greeks_results": "[{\\"date\\":\\"2025-09-01T00:00:00.000\\",\\"S\\":100,\\"K\\":105,\\"T\\":1.0,\\"r\\":0.05,\\"sigma\\":0.2,\\"option_type\\":\\"call\\",\\"price\\":8.0213522351,\\"delta\\":0.5422283336,\\"gamma\\":0.0198352619,\\"vega\\":39.6705238084,\\"rho\\":46.2014811233,\\"theta\\":-6.277126437},{\\"date\\":\\"2025-09-02T00:00:00.000\\",\\"S\\":102,\\"K\\":106,\\"T\\":0.9,\\"r\\":0.045,\\"sigma\\":0.19,\\"option_type\\":\\"put\\",\\"price\\":7.2142383713,\\"delta\\":-0.4596134147,\\"gamma\\":0.0215874826,\\"vega\\":38.4059448764,\\"rho\\":-48.685326005,\\"theta\\":-1.6196945478},{\\"date\\":\\"2025-09-03T00:00:00.000\\",\\"S\\":98,\\"K\\":104,\\"T\\":0.8,\\"r\\":0.048,\\"sigma\\":0.21,\\"option_type\\":\\"call\\",\\"price\\":6.415749359,\\"delta\\":0.4928141481,\\"gamma\\":0.0216695176,\\"vega\\":34.963159853,\\"rho\\":33.5040297211,\\"theta\\":-6.599156514},{\\"date\\":\\"2025-09-04T00:00:00.000\\",\\"S\\":101,\\"K\\":107,\\"T\\":0.7,\\"r\\":0.047,\\"sigma\\":0.18,\\"option

## Tool Level

In [ ]:
from bsm_multi_agents.tools import calculator_tools
importlib.reload(calculator_tools)
from bsm_multi_agents.tools.calculator_tools import batch_bsm_calculator, batch_greeks_calculator

## Agent Level

In [24]:
from bsm_multi_agents.agents.agent_factory import built_graph_agent_by_role
from bsm_multi_agents.graph.state import WorkflowState
from bsm_multi_agents.prompts.loader import load_prompt

from bsm_multi_agents.agents.data_loader_agent import data_loader_node

In [25]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
state = WorkflowState(csv_file_path=file_path)
state = data_loader_node(state)
prompt_path = os.path.join(project_path, "src/bsm_multi_agents/prompts/calculator_prompts.txt")

[data_loader] LLM: ChatOllama (qwen2.5:7b), Tools: ['csv_loader']


In [27]:
agent_role = "calculator"
default_system = """
You are a quantitative calculator agent.
"""
agent = built_graph_agent_by_role(agent_role,default_system=default_system)
user_prompt = load_prompt(prompt_path)
user_prompt

[calculator] LLM: ChatOllama (qwen2.5:7b), Tools: ['batch_bsm_calculator', 'batch_greeks_calculator', 'sensitivity_test']


'Calculate Black-Scholes-Merton option prices and greeks.\n- Use the `batch_bsm_calculator` tool to calculate prices.\n- Use the `batch_greeks_calculator` tool to calculate greeks.\n- The data has been loaded into the state.'

In [28]:
agent_input = state.copy()
agent_input["messages"] = [HumanMessage(content=user_prompt)]
agent_input["remaining_steps"] = 10
result = agent.invoke(
    agent_input,
    config={"recursion_limit": 10, "configurable": {"thread_id": state.get("thread_id","run-1")}}
)

In [29]:
print_resp(result)

Step 1 - inputs:
   Calculate Black-Scholes-Merton option prices and greeks.
- Use the `batch_bsm_calculator` tool to calculate prices.
- Use the `batch_greeks_calculator` tool to calculate greeks.
- The data has been lo...

Step 2 - Agent decide tools used:
   Tool name: batch_bsm_calculator
   Tool parameters: {}
   Tool name: batch_greeks_calculator
   Tool parameters: {}

Step 3 - outputs:
   Tool name: batch_bsm_calculator
   Outputs: {"bsm_results": "[{\"date\":\"2025-09-01T00:00:00.000\",\"S\":100,\"K\":105,\"T\":1.0,\"r\":0.05,\"sigma\":0.2,\"option_type\":\"call\",\"BSM_Price\":8.0213522351},{\"date\":\"2025-09-02T00:00:00.000\",\"S\":102,\"K\":106,\"T\":0.9,\"r\":0.045,\"sigma\":0.19,\"option_type\":\"put\",\"BSM_Price\":7.2142383713},{\"date\":\"2025-09-03T00:00:00.000\",\"S\":98,\"K\":104,\"T\":0.8,\"r\":0.048,\"sigma\":0.21,\"option_type\":\"call\",\"BSM_Price\":6.415749359},{\"date\":\"2025-09-04T00:00:00.000\",\"S\":101,\"K\":107,\"T\":0.7,\"r\":0.047,\"sigma\":0.18,\"op

In [33]:
bsm_result = get_tool_result_from_messages(result["messages"], "batch_bsm_calculator")

In [35]:
json.loads(bsm_result["result"])['bsm_results']

'[{"date":"2025-09-01T00:00:00.000","S":100,"K":105,"T":1.0,"r":0.05,"sigma":0.2,"option_type":"call","BSM_Price":8.0213522351},{"date":"2025-09-02T00:00:00.000","S":102,"K":106,"T":0.9,"r":0.045,"sigma":0.19,"option_type":"put","BSM_Price":7.2142383713},{"date":"2025-09-03T00:00:00.000","S":98,"K":104,"T":0.8,"r":0.048,"sigma":0.21,"option_type":"call","BSM_Price":6.415749359},{"date":"2025-09-04T00:00:00.000","S":101,"K":107,"T":0.7,"r":0.047,"sigma":0.18,"option_type":"call","BSM_Price":4.9529618176},{"date":"2025-09-05T00:00:00.000","S":99,"K":103,"T":0.6,"r":0.046,"sigma":0.22,"option_type":"put","BSM_Price":7.3776960425},{"date":"2025-09-06T00:00:00.000","S":103,"K":108,"T":0.5,"r":0.049,"sigma":0.2,"option_type":"put","BSM_Price":7.143363876},{"date":"2025-09-07T00:00:00.000","S":97,"K":102,"T":0.4,"r":0.044,"sigma":0.23,"option_type":"call","BSM_Price":4.2505156224},{"date":"2025-09-08T00:00:00.000","S":100,"K":106,"T":0.3,"r":0.05,"sigma":0.19,"option_type":"call","BSM_Price":

### main level

In [36]:
from bsm_multi_agents.agents.data_loader_agent import data_loader_node
from bsm_multi_agents.agents.calculator_agent import calculator_node

In [38]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
state = WorkflowState(csv_file_path=file_path)
out = data_loader_node(state)
state = {**state, **out}
out = calculator_node(state)
state = {**state, **out}
state

[data_loader] LLM: ChatOllama (qwen2.5:7b), Tools: ['csv_loader']
[calculator] LLM: ChatOllama (qwen2.5:7b), Tools: ['batch_bsm_calculator', 'batch_greeks_calculator', 'sensitivity_test']


{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
 'messages': [HumanMessage(content='Calculate Black-Scholes-Merton option prices and greeks.\n- Use the `batch_bsm_calculator` tool to calculate prices.\n- Use the `batch_greeks_calculator` tool to calculate greeks.\n- The data has been loaded into the state.', additional_kwargs={}, response_metadata={}, id='8fd36dc2-27b3-4ce7-a154-a57ae957c22f'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-11-25T21:21:04.401816Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1118738208, 'load_duration': 40940083, 'prompt_eval_count': 562, 'prompt_eval_duration': 627369333, 'eval_count': 39, 'eval_duration': 437778914, 'model_name': 'qwen2.5:7b', 'model_provider': 'ollama'}, id='lc_run--3d5d92e7-b286-493a-bf70-6d1d24fe8d27-0', tool_calls=[{'name': 'batch_bsm_calculator', 'args': {}, 'id': 'f7a7d972-064f-4136-913f-fbdfe4135949', 'type': 't

# Validator

In [10]:
from bsm_multi_agents.agents.data_loader_agent import data_loader_node
from bsm_multi_agents.agents.calculator_agent import calculator_node

In [11]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
state = WorkflowState(csv_file_path=file_path)
out = data_loader_node(state)
state = {**state, **out}
out = calculator_node(state)
state = {**state, **out}

[data_loader] LLM: ChatOllama (qwen2.5:7b), Tools: ['csv_loader']
[calculator] LLM: ChatOllama (qwen2.5:7b), Tools: ['batch_bsm_calculator', 'batch_greeks_calculator', 'sensitivity_test']


## Function Level

In [12]:
from bsm_multi_agents.tools import validator_tools
importlib.reload(validator_tools)
from bsm_multi_agents.tools.validator_tools import _validate_greeks_rules
from bsm_multi_agents.tools.utils import load_json_as_df

In [16]:
greeks_result = state.get("greeks_results")
df = load_json_as_df(greeks_result)
required_cols = ['option_type', 'price', 'delta', 'gamma', 'vega']
missing = [c for c in required_cols if c not in df.columns]
missing

[]

In [17]:
def calc_row(row):
    res = _validate_greeks_rules(
        row['option_type'], 
        row['price'], 
        row['delta'], 
        row['gamma'], 
        row['vega'], 
    )
    return res

expanded = df.apply(calc_row, axis=1).apply(pd.Series)
result_cols = ['validations_result','validations_details']
for col in result_cols:
    if col not in expanded:
        expanded[col] = pd.NA
df = pd.concat([df, expanded[result_cols]], axis=1)
result = {"validate_results": df.to_json(orient='records', date_format='iso')}
json.dumps(result)

'{"validate_results": "[{\\"date\\":\\"2025-09-01T00:00:00.000\\",\\"S\\":100,\\"K\\":105,\\"T\\":1.0,\\"r\\":0.05,\\"sigma\\":0.2,\\"option_type\\":\\"call\\",\\"price\\":8.0213522351,\\"delta\\":0.5422283336,\\"gamma\\":0.0198352619,\\"vega\\":39.6705238084,\\"rho\\":46.2014811233,\\"theta\\":-6.277126437,\\"validations_result\\":\\"passed\\",\\"validations_details\\":[]},{\\"date\\":\\"2025-09-02T00:00:00.000\\",\\"S\\":102,\\"K\\":106,\\"T\\":0.9,\\"r\\":0.045,\\"sigma\\":0.19,\\"option_type\\":\\"put\\",\\"price\\":7.2142383713,\\"delta\\":-0.4596134147,\\"gamma\\":0.0215874826,\\"vega\\":38.4059448764,\\"rho\\":-48.685326005,\\"theta\\":-1.6196945478,\\"validations_result\\":\\"passed\\",\\"validations_details\\":[]},{\\"date\\":\\"2025-09-03T00:00:00.000\\",\\"S\\":98,\\"K\\":104,\\"T\\":0.8,\\"r\\":0.048,\\"sigma\\":0.21,\\"option_type\\":\\"call\\",\\"price\\":6.415749359,\\"delta\\":0.4928141481,\\"gamma\\":0.0216695176,\\"vega\\":34.963159853,\\"rho\\":33.5040297211,\\"theta

## Tool Level

## Agent Level

In [27]:
from bsm_multi_agents.tools import validator_tools
importlib.reload(validator_tools)
from bsm_multi_agents.tools.validator_tools import batch_greeks_validator

from bsm_multi_agents.agents.agent_factory import built_graph_agent_by_role
from bsm_multi_agents.graph.state import WorkflowState
from bsm_multi_agents.prompts.loader import load_prompt

from bsm_multi_agents.agents.data_loader_agent import data_loader_node

In [28]:
agent_role = "validator"
default_system = """
You are a quantitative validator agent.
"""
agent = built_graph_agent_by_role(agent_role,default_system=default_system)

[validator] LLM: ChatOllama (qwen2.5:7b), Tools: ['batch_greeks_validator']


In [29]:
prompt_path = os.path.join(project_path, "src/bsm_multi_agents/prompts/validator_prompts.txt")
user_prompt = load_prompt(prompt_path)
user_prompt

'Run comprehensive validation tests on ALL options\n- Use the `batch_greeks_validator` tool to validate greeks.\n- The data has been loaded into the state.\n'

In [30]:
agent_input = state.copy()
agent_input["messages"] = [HumanMessage(content=user_prompt)]
agent_input["remaining_steps"] = 10
result = agent.invoke(
    agent_input,
    config={"recursion_limit": 10, "configurable": {"thread_id": state.get("thread_id","run-1")}}
)

In [33]:
validate_results = get_tool_result_from_messages(result["messages"], "batch_greeks_validator")

### main level

In [36]:
from bsm_multi_agents.agents.data_loader_agent import data_loader_node
from bsm_multi_agents.agents.calculator_agent import calculator_node
from bsm_multi_agents.agents.validator_agent import validator_node

In [37]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
state = WorkflowState(csv_file_path=file_path)
out = data_loader_node(state)
state = {**state, **out}
out = calculator_node(state)
state = {**state, **out}
out = validator_node(state)
state = {**state, **out}
state

[data_loader] LLM: ChatOllama (qwen2.5:7b), Tools: ['csv_loader']
[calculator] LLM: ChatOllama (qwen2.5:7b), Tools: ['batch_bsm_calculator', 'batch_greeks_calculator', 'sensitivity_test']
[validator] LLM: ChatOllama (qwen2.5:7b), Tools: ['batch_greeks_validator']


{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
 'messages': [HumanMessage(content='Run comprehensive validation tests on ALL options\n- Use the `batch_greeks_validator` tool to validate greeks.\n- The data has been loaded into the state.\n', additional_kwargs={}, response_metadata={}, id='92f84139-9c96-4963-b003-e7f8f7bfe401'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-11-25T22:58:32.059596Z', 'done': True, 'done_reason': 'stop', 'total_duration': 581993084, 'load_duration': 52341000, 'prompt_eval_count': 274, 'prompt_eval_duration': 312925750, 'eval_count': 19, 'eval_duration': 209160377, 'model_name': 'qwen2.5:7b', 'model_provider': 'ollama'}, id='lc_run--3cefed91-cd32-4c1f-b4c1-055e7d828e1e-0', tool_calls=[{'name': 'batch_greeks_validator', 'args': {}, 'id': 'ab5889f4-bb5d-4be4-a6bd-8ce52d0146ba', 'type': 'tool_call'}], usage_metadata={'input_tokens': 274, 'output_tokens':

# Summary Generator

In [43]:
from bsm_multi_agents.agents.data_loader_agent import data_loader_node
from bsm_multi_agents.agents.calculator_agent import calculator_node
from bsm_multi_agents.agents.validator_agent import validator_node

In [44]:
file_path = os.path.join(project_path, "data/input/dummy_options.csv")
state = WorkflowState(csv_file_path=file_path)
out = data_loader_node(state)
state = {**state, **out}
out = calculator_node(state)
state = {**state, **out}
out = validator_node(state)
state = {**state, **out}
state

[data_loader] LLM: ChatOllama (qwen2.5:7b), Tools: ['csv_loader']
[calculator] LLM: ChatOllama (qwen2.5:7b), Tools: ['batch_bsm_calculator', 'batch_greeks_calculator', 'sensitivity_test']
[validator] LLM: ChatOllama (qwen2.5:7b), Tools: ['batch_greeks_validator']


{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
 'messages': [HumanMessage(content='Run comprehensive validation tests on ALL options\n- Use the `batch_greeks_validator` tool to validate greeks.\n- The data has been loaded into the state.\n', additional_kwargs={}, response_metadata={}, id='e4f3bfb3-4c46-49c5-b67a-98dcd21352f3'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-11-25T23:06:55.699311Z', 'done': True, 'done_reason': 'stop', 'total_duration': 579758292, 'load_duration': 50531750, 'prompt_eval_count': 274, 'prompt_eval_duration': 312057834, 'eval_count': 19, 'eval_duration': 209649792, 'model_name': 'qwen2.5:7b', 'model_provider': 'ollama'}, id='lc_run--c3ea1efb-87fa-49ad-977b-0aefc543be0b-0', tool_calls=[{'name': 'batch_greeks_validator', 'args': {}, 'id': 'bd7c5c48-c58a-40cf-8738-00807bce9b67', 'type': 'tool_call'}], usage_metadata={'input_tokens': 274, 'output_tokens':

## Function Level

In [101]:
from bsm_multi_agents.tools import summary_generator_tools
importlib.reload(summary_generator_tools)
from bsm_multi_agents.tools.summary_generator_tools import (
    _parse_input_data,
    _compute_market_stats,
    _compute_greek_stats,
    _compute_bsm_stats,
    _generate_recommendations,
    _load_template,
)

In [81]:
template_path = os.path.join(project_path, "src/bsm_multi_agents/templates/summary_template.md")

In [83]:
bsm_df, greeks_df, vr_df = _parse_input_data(state)

In [89]:
total = len(vr_df)
pass_cnt = int((vr_df.get("validations_result") == "passed").sum()) if "validations_result" in vr_df else 0
fail_cnt = total - pass_cnt
pass_rate = f"{(pass_cnt / total * 100):.1f}" if total > 0 else "0.0"

option_types_inline = "N/A"
if "option_type" in vr_df.columns:
    counts = vr_df["option_type"].value_counts().to_dict()
    option_types_inline = ", ".join([f"{k}: {v}" for k, v in counts.items()])

issues = []
if "validations_details" in vr_df.columns:
    raw_issues = vr_df[vr_df["validations_result"] == "failed"]["validations_details"].tolist()
    for item in raw_issues:
        if isinstance(item, list):
            issues.extend([str(i) for i in item if i])
        elif item:
            issues.append(str(item))
critical_issues = "\n".join([f"- {i}" for i in issues[:10]]) if issues else "None"
if len(issues) > 10:
    critical_issues += f"\n- ... and {len(issues) - 10} more."


In [98]:
market_stats = _compute_market_stats(vr_df)
greek_stats = _compute_greek_stats(vr_df)
context = {
            "analysis_date": datetime.now().strftime("%Y-%m-%d"),
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "total_options": total,
            "validation_status": "✅ PASSED" if fail_cnt == 0 else "⚠️ ISSUES FOUND",
            "pass_rate": pass_rate,
            "failed_count": fail_cnt,
            "option_types_inline": option_types_inline,
            "bsm_pricing_summary": _compute_bsm_stats(bsm_df),
            "critical_issues": critical_issues,
            "recommendations": _generate_recommendations(fail_cnt, vr_df),
            **market_stats,
            **greek_stats
        }

In [102]:
template_txt = _load_template(template_path)
report_md = template_txt.format(**context)

In [103]:
report_md

'# 📊 BSM Model Ongoing Performance Analysis\n\n**Date:** 2025-11-25 | **Total Options:** 10 | **Status:** ✅ PASSED\n\n## 1. Executive Summary\n\n| Metric | Value |\n| :--- | :--- |\n| **Total Processed** | 10 |\n| **Pass Rate** | 100.0% |\n| **Failed** | 0 |\n| **Option Types** | call: 5, put: 5 |\n\n## 2. Market Data Overview\n\n| Metric | Range | Average |\n| :--- | :--- | :--- |\n| **Spot Price (S)** | [96.00, 104.00] | 100.00 |\n| **Strike Price (K)** | [101.00, 109.00] | 105.10 |\n| **Maturity (T)** | [0.10, 1.00] | 0.55 |\n| **Volatility (σ)** | [18.00%, 23.00%] | 20.30% |\n\n## 3. Pricing & Greeks Analysis\n\n### BSM Pricing\n- **Total Priced:** 10\n- **Avg Price:** $5.9433\n- **Range:** [$2.3917, $8.0214]\n- **Total Value:** $59.43\n\n### Greeks Profile\n| Greek | Average | Range | Expected |\n| :--- | :--- | :--- | :--- |\n| **Delta** | -0.0589 | [-0.7566, 0.5422] | [-1, 1] |\n| **Gamma** | 0.0294 | [0.0198, 0.0516] | ≥ 0 |\n| **Vega** | 27.7706 | [9.5107, 39.6705] | ≥ 0 |\n| 

## Agent Level